# Data visualization Project

## Part 3

In [2]:
#import the essential packages
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, HBox, VBox
import numpy as np
from bqplot import *
from bqplot import Tooltip, pyplot, marks
import pandas as pd
import us
from IPython.display import display

import traitlets

#little bit of magic!
%matplotlib inline

In [10]:
#import the csv file ufo-scrubbed-geocoded-time-standardized
names = ["date", "city", "state", "country", "shape", "duration_seconds", "duration_reported", "description", "report_date", "latitude", "longitude"]

ufo = pd.read_csv("ufo-scrubbed-geocoded-time-standardized.csv",
                  names = names, parse_dates = ["date", "report_date"])
#Map abbr to fips
abbr_to_fips = us.states.mapping('abbr', 'fips')

#create new column in ufo dataframe, with fips code!
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fips.get(str(a).upper(), -1)))

#Total time found out by grouping by states(in the form of fips) so that value is for each state
total_time = ufo.groupby(["fips"])["duration_seconds"].sum()

#Total sightings found out by grouping by states(in the form of fips) so that value is for each state
total_sightings = ufo.groupby(["fips"])["report_date"].count()

In [11]:
#creating dataframe with columns total_sightings and total_time
df = pd.DataFrame([total_sightings, total_time])

#Transposing it
df = df.T

#Giving columns values
df.columns = ['total sightings', 'total time']

#removing first row
df = df[1:]

In [4]:
type(total_sightings)

pandas.core.series.Series

In [12]:
#Adding new column named year in the ufo dataframe object by parsing and finding year in ufo['date']
ufo['year'] = pd.DatetimeIndex(ufo['date']).year

In [79]:

def showPlot(change):
    global ch
    print(type(change['new']))


    if len(change.new) > 1:
        states_map.selected = [change.new[-1]]
    elif len(change.new) == 1:
        print("yay")
    
        ch = states_map.selected[-1:]
        ufo_state = ufo[ufo['fips'] == ch[0]]

        print(states_map.selected[-1:])
        scatter_plot = interact(getCorresPlot, year = widgets.IntRangeSlider(
                    value=[ufo_state['year'].max() - 50,ufo_state['year'].max()],
                    min=ufo_state['year'].min(),
                    max=ufo_state['year'].max(),
                    continuous_update=False,
                    step=10,
                ));
        


def getCorresPlot(year):   
    
    print(year[0])
    print(year[1])
    print(ch)
    ufo_state = ufo[ufo['fips'] == ch[0]]
    #ch[:] = []
    ufo_plot = ufo_state[ufo_state['year'] > year[0]]
    
    total_time_year = ufo_state.groupby(["year"])["duration_seconds"].sum() 
    total_sightings_year = ufo_state.groupby(["year"])["report_date"].count()
    
    sc_x = LinearScale()
    sc_y = LinearScale()
    scatter = Scatter(x=ufo_plot['year'].unique(), y=total_sightings_year, scales={'x': sc_x, 'y': sc_y}, colors=['blue'])

    ax_x = Axis(label='Year', scale=sc_x)
    ax_y = Axis(label='Test Y', scale=sc_y, orientation='vertical', tick_format='0.2f')

    fig = Figure(axes=[ax_x, ax_y], marks=[scatter])
    
    return fig
    

        

def getMap(mode):
    if mode == 'total sightings':
        def_tt = Tooltip(fields=['name', 'color'], labels = ['state', 'total_sightings_year'])
    else:
        def_tt = Tooltip(fields=['name', 'color'], labels = ['state', 'total_time_year'])
        
    global states_map
    
    map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                             'color': bqplot.ColorScale(colors=["brown", "blue", "red"])},
                  'color': df[mode].to_dict()}
    states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'), **map_styles, tooltip=def_tt
                           )

    states_map.interactions = {'click': 'select', 'hover': 'tooltip'}
    #states_map.observe(getCorresPlot)
    #display(getCorresPlot)
    
    #states_map.observe(showPlot, names= ['selected'])
    map_figure = bqplot.Figure(marks=[states_map], fig_margin={'top':30, 'bottom':30, 'left':0, 'right':0})
    return map_figure


map_figure = interact(getMap, mode = widgets.Dropdown(
                                options=['total sightings', 'total time'],
                                value='total sightings',
                                disabled=False,
                            ));






states_map.observe(showPlot, names= ['selected'])

interactive(children=(Dropdown(description='mode', options=('total sightings', 'total time'), value='total sightings'), Output()), _dom_classes=('widget-interact',))

In [4]:
census_data = pd.read_csv("census_state_quickfacts.csv")
census_data = census_data.transpose()

census_data = census_data[census_data.columns[:2]]
census_data.head()


,0,1
People,Population,Population estimates; July 1; 2016; (V2016)
Alaska,NaN,741894
Georgia,NaN,10310371
Hawaii,NaN,1428557
Idaho,NaN,1683140


In [5]:
del census_data[census_data.columns[0]]

In [6]:
census_data.head()

,1
People,Population estimates; July 1; 2016; (V2016)
Alaska,741894
Georgia,10310371
Hawaii,1428557
Idaho,1683140


In [9]:
#Population of each state

state_code = pd.read_csv("state_code.csv", delimiter='|')
state_code

,STATE,STUSAB,STATE_NAME,STATENS
0,1,AL,Alabama,1779775
1,2,AK,Alaska,1785533
2,4,AZ,Arizona,1779777
3,5,AR,Arkansas,68085
4,6,CA,California,1779778
5,8,CO,Colorado,1779779
6,9,CT,Connecticut,1779780
7,10,DE,Delaware,1779781
8,11,DC,District of Columbia,1702382
9,12,FL,Florida,294478
